```{contents}
```
## **Override in LangGraph**

In LangGraph, an **Override** is a controlled mechanism that allows **external intervention** to modify the execution of a running graph by **altering its state, routing decisions, or execution flow**, typically used for **human-in-the-loop systems, error recovery, governance, and production safety**.

Overrides make LangGraph **adaptable and safe**, enabling operators or higher-level controllers to **steer, correct, or halt autonomous behavior** without rebuilding the graph.

---

### **1. Why Overrides Are Necessary**

Autonomous LLM workflows are **probabilistic and non-deterministic**.
Overrides provide:

| Need                | Purpose                        |
| ------------------- | ------------------------------ |
| Error correction    | Fix wrong intermediate results |
| Safety control      | Stop harmful behavior          |
| Human supervision   | Enforce governance             |
| Policy enforcement  | Apply business rules           |
| Operational control | Handle unexpected states       |
| Debugging           | Modify execution mid-run       |

---

### **2. Conceptual Model**

```
        Normal Execution
State ───────────▶ Node ───────────▶ State

           ▲
           │
      External Override
           │
           ▼

State (modified) ───────▶ Execution continues
```

An override **injects a new version of state** or **redirects control flow** before execution proceeds.

---

### **3. What Can Be Overridden**

| Element               | Description                      |
| --------------------- | -------------------------------- |
| Graph State           | Modify any state field           |
| Routing Decision      | Change next node                 |
| Execution Flow        | Skip, repeat, or terminate nodes |
| Tool Result           | Replace tool output              |
| Agent Output          | Replace LLM response             |
| Termination Condition | Force stop or continue           |

---

### **4. Override via State Injection**

Overrides are typically implemented by **manipulating the shared state**.

```python
def human_override(state):
    state["answer"] = "Corrected by human"
    state["approved"] = True
    return state
```

Inserted as a **human node**:

```python
builder.add_node("override", human_override)
builder.add_edge("analyze", "override")
builder.add_edge("override", "finalize")
```

---

### **5. Conditional Overrides (Policy-Based)**

```python
def policy_router(state):
    if state["risk"] > 0.8:
        return "override"
    return "continue"
```

```python
builder.add_conditional_edges("check", policy_router, {
    "override": "override",
    "continue": "execute"
})
```

This creates **automatic overrides** when safety conditions trigger.

---

### **6. Runtime Overrides (External Intervention)**

Using **interrupts & checkpoints**, operators can override during execution:

```python
graph.invoke(input, config={"interrupt_after": ["analyze"]})
```

Load checkpoint → modify state → resume:

```python
checkpoint = store.load(thread_id)
checkpoint["approved"] = False
checkpoint["answer"] = "Manually fixed"
graph.invoke(checkpoint)
```

---

### **7. Override Patterns**

| Pattern               | Use Case             |
| --------------------- | -------------------- |
| Human Approval Gate   | Compliance, legal    |
| Emergency Kill-Switch | Safety shutdown      |
| Correction Injection  | Fix LLM errors       |
| Policy Enforcement    | Regulatory control   |
| Recovery Override     | Failure handling     |
| Debug Override        | Testing & inspection |

---

### **8. Production Safety Guarantees**

| Control          | Benefit               |
| ---------------- | --------------------- |
| Audit logging    | Full traceability     |
| Checkpointing    | Safe rollback         |
| Versioned state  | Change tracking       |
| Access control   | Who can override      |
| Timeouts         | Prevent deadlock      |
| Recursion limits | Prevent runaway loops |

---

### **9. Comparison with Hardcoded Logic**

| Without Override | With Override     |
| ---------------- | ----------------- |
| Rigid            | Adaptive          |
| Unsafe autonomy  | Governed autonomy |
| Hard to debug    | Fully observable  |
| No human control | Human-in-the-loop |

---

### **10. Key Insight**

> **Overrides transform LangGraph from an autonomous engine into a controlled, production-grade AI system.**

They allow **human judgment, policy enforcement, and operational safety** to coexist with **LLM autonomy**.


### Demonstration

In [1]:
# One-cell LangGraph Override Demonstration
# ---------------------------------------
# This example shows:
# - normal execution
# - conditional policy override
# - human-style correction injection
# - safe termination

from typing import TypedDict
from langgraph.graph import StateGraph, END

# ----- 1. State Definition -----
class State(TypedDict):
    text: str
    risk: float
    answer: str
    approved: bool

# ----- 2. Normal Processing Node -----
def analyze(state: State):
    return {
        "answer": f"LLM output for: {state['text']}",
        "risk": 0.9,  # simulate high-risk result
        "approved": False
    }

# ----- 3. Policy Router -----
def policy_router(state: State):
    if state["risk"] > 0.7:
        return "override"
    return "finalize"

# ----- 4. Override Node (Human / Policy Intervention) -----
def override(state: State):
    return {
        "answer": "Corrected by override: Safe, approved response",
        "approved": True,
        "risk": 0.0
    }

# ----- 5. Final Node -----
def finalize(state: State):
    return {"answer": state["answer"]}

# ----- 6. Build Graph -----
builder = StateGraph(State)

builder.add_node("analyze", analyze)
builder.add_node("override", override)
builder.add_node("finalize", finalize)

builder.set_entry_point("analyze")

builder.add_conditional_edges("analyze", policy_router, {
    "override": "override",
    "finalize": "finalize"
})

builder.add_edge("override", "finalize")
builder.add_edge("finalize", END)

graph = builder.compile()

# ----- 7. Run -----
result = graph.invoke({
    "text": "Generate financial advice",
    "risk": 0.0,
    "answer": "",
    "approved": False
})

print(result)


{'text': 'Generate financial advice', 'risk': 0.0, 'answer': 'Corrected by override: Safe, approved response', 'approved': True}
